### AB Model from spec

Package installation

In [41]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [42]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
from datetime import datetime
import logging
# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [43]:
import model_config as cfg
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

State diagram

### Model

Model parameters

In [44]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 5
# no of css must be fixed
no_css = 5

Run Model

In [45]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css)
for i in range(ticks):
    model_run.step()


EV info: ID: 0, destination name: work, journey type: Urban, max_battery: 53, energy consumption rate: 0.2, speed: 10, State: Idle.
EV info (Cont'd): Start time: 12, distance goal: 50, soc usage threshold: 26.5, charge prop 0.5, location: City_D.

EV info: ID: 1, destination name: City A, journey type: InterUrban, max_battery: 55, energy consumption rate: 0.5, speed: 20, State: Idle.
EV info (Cont'd): Start time: 6, distance goal: 120, soc usage threshold: 27.5, charge prop 0.5, location: City_D.

EV info: ID: 2, destination name: friend_1, journey type: Urban, max_battery: 68, energy consumption rate: 0.2, speed: 10, State: Idle.
EV info (Cont'd): Start time: 14, distance goal: 30, soc usage threshold: 34.0, charge prop 0.5, location: City_D.

EV info: ID: 3, destination name: friend_1, journey type: Urban, max_battery: 63, energy consumption rate: 0.2, speed: 10, State: Idle.
EV info (Cont'd): Start time: 11, distance goal: 30, soc usage threshold: 31.5, charge prop 0.5, location: C

In [46]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charging  EVs Activated  EVs Travelling  EVs Queued  EVs Dead  \
0              0              5               0           0         0   
1              0              5               0           0         0   
2              0              5               0           0         0   
3              0              5               0           0         0   
4              0              5               0           0         0   
5              0              5               0           0         0   
6              0              5               1           0         0   
7              0              5               2           0         0   
8              0              5               2           0         0   
9              1              5               1           0         0   
10             1              5               1           0         0   
11             1              5               2           0         0   
12             2              5               2    

In [47]:
a = 26 % 24
print(a) 

2


Export results to CSV file

In [74]:
date_str = str(datetime.today())
model_run.datacollector.get_model_vars_dataframe().to_csv(cfg.DATA_PATH + '_' + date_str[0:10] + '_' + str(no_evs) + '_EV_agent_model_output.csv')

### Data import and preprocessing

Data import and Helper functions

In [76]:
import analysis.helper as helper
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [51]:
data = pd.read_csv(cfg.DATA_PATH +'/06_03_5EV_agent_model_output.csv')

In [52]:
# data.info()
data.head()

,Unnamed: 0,EVs Charging,EVs Activated,EVs Travelling,EVs Queued,EVs Dead,EVs Charge Level,EVs Charge Propensity,EVs Not Idle,EVs EOD Battery SOC,EVs Destinations,EVs at Charging Station - S,Length of Queue 1 at Charging Stations,Length of Queue 2 at Charging Stations,EVs at Charging Stations
0,0,0,2,0,0,0,"[63, 48]","[0.5, 0.5]",0,"[[63, 63], [48, 48]]","['friend_2', 'market']",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
1,1,0,2,0,0,0,"[63, 48]","[0.5, 0.5]",0,"[[63, 63], [48, 48]]","['friend_2', 'market']",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
2,2,0,2,0,0,0,"[63, 48]","[0.5, 0.5]",0,"[[63, 63], [48, 48]]","['friend_2', 'market']",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
3,3,0,2,0,0,0,"[63, 48]","[0.5, 0.5]",0,"[[63, 63], [48, 48]]","['friend_2', 'market']",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
4,4,0,2,0,0,0,"[63, 48]","[0.5, 0.5]",0,"[[63, 63], [48, 48]]","['friend_2', 'market']",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN


In [75]:
# a = helper.unpack_and_join(data, 'EVs Charge Level')
print(a)

2023-03-13 13:26:26.889572


In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

KeyError: "['EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'] not in index"

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

### Scrapbook